# Translation (PyTorch)

Install the Transformers, Datasets, and Evaluate libraries to run this notebook.

In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
!apt install git-lfs
!pip install -q kagglehub transformers torch accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 45.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 57.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling 

You will need to setup git, adapt your email and name in the following cell.

In [2]:
!git config --global user.email "joao.paulo.migliatti@gmail.com"
!git config --global user.name "psicanalise10"

You will also need to be logged in to the Hugging Face Hub. Execute the following and enter your credentials.

In [33]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
from google.colab import files
uploaded = files.upload()  # Selecione o kaggle.json baixado do site

Saving kaggle.json to kaggle.json


In [5]:
import kagglehub

# 🔽 Baixar dataset (isso retorna o caminho completo)
path = kagglehub.dataset_download("djonafegnem/europarl-parallel-corpus-19962011")

# 🔍 Ver arquivos disponíveis no diretório baixado
import os
print("Arquivos disponíveis:", os.listdir(path))


100%|██████████| 2.37G/2.37G [00:29<00:00, 87.5MB/s]

Extracting files...


Arquivos disponíveis: ['english_french.csv', 'english_spanish.csv', 'english_dutch.csv', 'english_czech.csv', 'english_estonian.csv', 'english_slovene.csv', 'english_lithunian.csv', 'english_greek.csv', 'english_polish.csv', 'english_bulgarian.csv', 'english_finnish.csv', 'english_hungarian.csv', 'english_latvia.csv', 'english_romanian.csv', 'english_german.csv', 'english_slovak.csv', 'english_italian.csv', 'english_swedish.csv', 'english_portuguese.csv']


In [6]:
import pandas as pd

df = pd.read_csv(os.path.join(path, "english_portuguese.csv"))
df = df.dropna().reset_index(drop=True)
df = df.rename(columns={"English": "en", "Portuguese": "pt"})


In [14]:
import os
import pandas as pd
from datasets import Dataset

# Carrega apenas os primeiros 5.000 exemplos
df = pd.read_csv(os.path.join(path, "english_portuguese.csv")).dropna().reset_index(drop=True)
df = df.rename(columns={"English": "en", "Portuguese": "pt"})
df = df.sample(frac=1, random_state=42).reset_index(drop=True)
df = df[:10000]


raw_datasets = Dataset.from_pandas(df[["en", "pt"]])
raw_datasets = raw_datasets.train_test_split(test_size=0.2)


In [12]:
raw_datasets

Dataset({
    features: ['en', 'pt'],
    num_rows: 10000
})

In [15]:
split_datasets = raw_datasets["train"].train_test_split(train_size=0.9, seed=20)
split_datasets

DatasetDict({
    train: Dataset({
        features: ['en', 'pt'],
        num_rows: 7200
    })
    test: Dataset({
        features: ['en', 'pt'],
        num_rows: 800
    })
})

In [16]:
split_datasets["validation"] = split_datasets.pop("test")

In [24]:
from datasets import DatasetDict

def format_translation(example):
    return {"translation": {"en": example["en"], "pt": example["pt"]}}

formatted_train = split_datasets["train"].map(format_translation)

formatted_validation = split_datasets["validation"].map(format_translation)

split_datasets = DatasetDict({
    "train": formatted_train,
    "validation": formatted_validation
})


Map:   0%|          | 0/7200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [25]:
split_datasets["train"][1]["translation"]

{'en': 'It is clear that this is not just an economic and financial crisis, but also one of ethics and responsibility.',
 'pt': 'Parece claro que não se trata apenas de uma crise económica e financeira, trata-se também de uma crise de ética e responsabilidade.'}

In [26]:
from transformers import pipeline

model_checkpoint = "Helsinki-NLP/opus-mt-en-fr"
translator = pipeline("translation", model=model_checkpoint)
translator("Default to expanded threads")

config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/301M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/301M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/778k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/802k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:177: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
Device set to use cpu


[{'translation_text': 'Par défaut pour les threads élargis'}]

In [27]:
split_datasets["train"][172]["translation"]

{'en': 'In this way, we shall never see a positive opinion from the Court of Auditors.',
 'pt': 'A agir desta forma nunca iremos conseguir um parecer positivo por parte do Tribunal de Contas.'}

In [30]:
translator(
    "Unable to import %1 using the OFX importer plugin. This file is not the correct format."
)

[{'translation_text': "Impossible d'importer %1 en utilisant le plugin d'importateur OFX. Ce fichier n'est pas le bon format."}]

In [31]:
from transformers import AutoTokenizer

model_checkpoint = "Helsinki-NLP/opus-mt-tc-bible-big-mul-mul"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, return_tensors="pt")

tokenizer_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

In [35]:
en_sentence = split_datasets["train"][1]["translation"]["en"]
pt_sentence = split_datasets["train"][1]["translation"]["pt"]

inputs = tokenizer(en_sentence, text_target=pt_sentence)
inputs

{'input_ids': [32745, 44918, 39298, 55737, 55758, 44918, 49450, 45386, 37034, 41237, 37059, 42349, 39869, 50, 38605, 36930, 50106, 49923, 41874, 10777, 15831, 37059, 53010, 84, 524], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [34282, 39289, 52436, 49640, 53697, 56192, 37217, 40239, 56815, 39868, 41242, 41208, 42345, 50, 56192, 55, 15952, 55374, 40239, 56815, 39868, 40239, 29693, 19100, 41208, 53005, 9523, 84, 524]}

In [36]:
wrong_targets = tokenizer(pt_sentence)
print(tokenizer.convert_ids_to_tokens(wrong_targets["input_ids"]))
print(tokenizer.convert_ids_to_tokens(inputs["labels"]))

['▁Parece', '▁claro', '▁que', '▁não', '▁se', '▁trata', '▁apenas', '▁de', '▁uma', '▁crise', '▁económica', '▁e', '▁financeira', ',', '▁trata', '-', 'se', '▁também', '▁de', '▁uma', '▁crise', '▁de', '▁', 'ética', '▁e', '▁responsabilidad', 'e', '.', '</s>']
['▁Parece', '▁claro', '▁que', '▁não', '▁se', '▁trata', '▁apenas', '▁de', '▁uma', '▁crise', '▁económica', '▁e', '▁financeira', ',', '▁trata', '-', 'se', '▁também', '▁de', '▁uma', '▁crise', '▁de', '▁', 'ética', '▁e', '▁responsabilidad', 'e', '.', '</s>']


In [37]:
max_length = 128


def preprocess_function(examples):
    inputs = [ex["en"] for ex in examples["translation"]]
    targets = [ex["pt"] for ex in examples["translation"]]
    model_inputs = tokenizer(
        inputs, text_target=targets, max_length=max_length, truncation=True
    )
    return model_inputs

In [38]:
tokenized_datasets = split_datasets.map(
    preprocess_function,
    batched=True,
    remove_columns=split_datasets["train"].column_names,
)

Map:   0%|          | 0/7200 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

In [39]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/991M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/292 [00:00<?, ?B/s]

In [40]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [41]:
batch = data_collator([tokenized_datasets["train"][i] for i in range(1, 3)])
batch.keys()

dict_keys(['input_ids', 'attention_mask', 'labels', 'decoder_input_ids'])

In [42]:
batch["labels"]

tensor([[34282, 39289, 52436, 49640, 53697, 56192, 37217, 40239, 56815, 39868,
         41242, 41208, 42345,    50, 56192,    55, 15952, 55374, 40239, 56815,
         39868, 40239, 29693, 19100, 41208, 53005,  9523,    84,   524],
        [32977, 49640, 53020, 48361, 55611, 59141, 35692, 15283, 50538, 39945,
         15052, 15311, 50279, 39877, 51276, 15831, 40239, 31197, 14819, 10647,
         10237,    84,   524,  -100,  -100,  -100,  -100,  -100,  -100]])

In [43]:
batch["decoder_input_ids"]

tensor([[69666, 34282, 39289, 52436, 49640, 53697, 56192, 37217, 40239, 56815,
         39868, 41242, 41208, 42345,    50, 56192,    55, 15952, 55374, 40239,
         56815, 39868, 40239, 29693, 19100, 41208, 53005,  9523,    84],
        [69666, 32977, 49640, 53020, 48361, 55611, 59141, 35692, 15283, 50538,
         39945, 15052, 15311, 50279, 39877, 51276, 15831, 40239, 31197, 14819,
         10647, 10237,    84,   524, 69666, 69666, 69666, 69666, 69666]])

In [44]:
for i in range(1, 3):
    print(tokenized_datasets["train"][i]["labels"])

[34282, 39289, 52436, 49640, 53697, 56192, 37217, 40239, 56815, 39868, 41242, 41208, 42345, 50, 56192, 55, 15952, 55374, 40239, 56815, 39868, 40239, 29693, 19100, 41208, 53005, 9523, 84, 524]
[32977, 49640, 53020, 48361, 55611, 59141, 35692, 15283, 50538, 39945, 15052, 15311, 50279, 39877, 51276, 15831, 40239, 31197, 14819, 10647, 10237, 84, 524]


In [45]:
!pip install sacrebleu

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.5 MB/s eta 0:00:00


In [46]:
import evaluate

metric = evaluate.load("sacrebleu")

In [47]:
predictions = [
    "This plugin lets you translate web pages between several languages automatically."
]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 46.750469682990165,
 'counts': [11, 6, 4, 3],
 'totals': [12, 11, 10, 9],
 'precisions': [91.66666666666667,
  54.54545454545455,
  40.0,
  33.333333333333336],
 'bp': 0.9200444146293233,
 'sys_len': 12,
 'ref_len': 13}

In [48]:
predictions = ["This This This This"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 1.683602693167689,
 'counts': [1, 0, 0, 0],
 'totals': [4, 3, 2, 1],
 'precisions': [25.0, 16.666666666666668, 12.5, 12.5],
 'bp': 0.10539922456186433,
 'sys_len': 4,
 'ref_len': 13}

In [49]:
predictions = ["This plugin"]
references = [
    [
        "This plugin allows you to automatically translate web pages between several languages."
    ]
]
metric.compute(predictions=predictions, references=references)

{'score': 0.0,
 'counts': [2, 1, 0, 0],
 'totals': [2, 1, 0, 0],
 'precisions': [100.0, 100.0, 0.0, 0.0],
 'bp': 0.004086771438464067,
 'sys_len': 2,
 'ref_len': 13}

In [50]:
import numpy as np


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    # In case the model returns more than the prediction logits
    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100s in the labels as we can't decode them
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    return {"bleu": result["score"]}

In [51]:
from huggingface_hub import notebook_login

notebook_login()

In [54]:
from transformers import Seq2SeqTrainingArguments

args = Seq2SeqTrainingArguments(
    f"joao-finetuned-kde4-en-to-pt",
    save_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=3,
    predict_with_generate=True,
    fp16=True,
    push_to_hub=True,
)

In [55]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

/tmp/ipython-input-55-543751272.py:3: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


In [56]:
trainer.evaluate(max_length=max_length)

KeyboardInterrupt: 

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate(max_length=max_length)

In [ ]:
trainer.push_to_hub(tags="translation", commit_message="Training complete")

In [ ]:
from torch.utils.data import DataLoader

tokenized_datasets.set_format("torch")
train_dataloader = DataLoader(
    tokenized_datasets["train"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
eval_dataloader = DataLoader(
    tokenized_datasets["validation"], collate_fn=data_collator, batch_size=8
)

In [ ]:
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [ ]:
from transformers import AdamW

optimizer = AdamW(model.parameters(), lr=2e-5)

In [ ]:
from accelerate import Accelerator

accelerator = Accelerator()
model, optimizer, train_dataloader, eval_dataloader = accelerator.prepare(
    model, optimizer, train_dataloader, eval_dataloader
)

In [ ]:
from transformers import get_scheduler

num_train_epochs = 3
num_update_steps_per_epoch = len(train_dataloader)
num_training_steps = num_train_epochs * num_update_steps_per_epoch

lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps,
)

In [ ]:
from huggingface_hub import Repository, get_full_repo_name

model_name = "joao-finetuned-kde4-en-to-pt-accelerate"
repo_name = get_full_repo_name(model_name)
repo_name

In [ ]:
output_dir = "joao-finetuned-kde4-en-to-pt-accelerate"
repo = Repository(output_dir, clone_from=repo_name)

In [ ]:
def postprocess(predictions, labels):
    predictions = predictions.cpu().numpy()
    labels = labels.cpu().numpy()

    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds = [pred.strip() for pred in decoded_preds]
    decoded_labels = [[label.strip()] for label in decoded_labels]
    return decoded_preds, decoded_labels

In [ ]:
from tqdm.auto import tqdm
import torch

progress_bar = tqdm(range(num_training_steps))

for epoch in range(num_train_epochs):
    # Training
    model.train()
    for batch in train_dataloader:
        outputs = model(**batch)
        loss = outputs.loss
        accelerator.backward(loss)

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()
        progress_bar.update(1)

    # Evaluation
    model.eval()
    for batch in tqdm(eval_dataloader):
        with torch.no_grad():
            generated_tokens = accelerator.unwrap_model(model).generate(
                batch["input_ids"],
                attention_mask=batch["attention_mask"],
                max_length=128,
            )
        labels = batch["labels"]

        # Necessary to pad predictions and labels for being gathered
        generated_tokens = accelerator.pad_across_processes(
            generated_tokens, dim=1, pad_index=tokenizer.pad_token_id
        )
        labels = accelerator.pad_across_processes(labels, dim=1, pad_index=-100)

        predictions_gathered = accelerator.gather(generated_tokens)
        labels_gathered = accelerator.gather(labels)

        decoded_preds, decoded_labels = postprocess(predictions_gathered, labels_gathered)
        metric.add_batch(predictions=decoded_preds, references=decoded_labels)

    results = metric.compute()
    print(f"epoch {epoch}, BLEU score: {results['score']:.2f}")

    # Save and upload
    accelerator.wait_for_everyone()
    unwrapped_model = accelerator.unwrap_model(model)
    unwrapped_model.save_pretrained(output_dir, save_function=accelerator.save)
    if accelerator.is_main_process:
        tokenizer.save_pretrained(output_dir)
        repo.push_to_hub(
            commit_message=f"Training in progress epoch {epoch}", blocking=False
        )

In [ ]:
from transformers import pipeline

# Replace this with your own checkpoint
model_checkpoint = "huggingface-course/marian-finetuned-kde4-en-to-fr"
translator = pipeline("translation", model=model_checkpoint)
translator("Default to expanded threads")

In [ ]:
translator(
    "Unable to import %1 using the OFX importer plugin. This file is not the correct format."
)